In [1]:
%load_ext autoreload

In [5]:
from rnapolis.annotator import extract_secondary_structure
from rnapolis.parser import read_3d_structure

pdb_path = "/data/3d/input_data/pdbs/2A43_1_A.pdb"

# read 3d structure
with open(pdb_path) as f:
    structure3d = read_3d_structure(f, 1)
    structure2d = extract_secondary_structure(structure3d, 1)

interest_res = []
interest_bps = []
for bp in structure2d.basePairs:
    bp_type = bp.saenger.is_canonical if bp.saenger is not None else False
    print(bp_type, bp.nt1, bp.nt2)
    if bp_type and bp.nt1.name == 'G' or bp.nt2.name == 'G':
        print(f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}")
        print(f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}")
        bp1 = f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}"
        bp2 = f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}"
        interest_res.append(bp1)
        interest_res.append(bp2)
        interest_bps.append((bp1, bp2))
    # if not bp_type:
    #     print(bp_type, bp.nt1, bp.nt2)
    #     print(f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}")
    #     print(f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}")
    #     bp1 = f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}"
    #     bp2 = f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}"
    #     interest_res.append(bp1)
    #     interest_res.append(bp2)
    #     interest_bps.append((bp1, bp2))

True Residue(label=None, auth=ResidueAuth(chain='A', number=3, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=16, icode=None, name='C'))
A.G3
A.C16
False Residue(label=None, auth=ResidueAuth(chain='A', number=3, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=20, icode=None, name='A'))
True Residue(label=None, auth=ResidueAuth(chain='A', number=4, icode=None, name='C')) Residue(label=None, auth=ResidueAuth(chain='A', number=15, icode=None, name='G'))
A.C4
A.G15
True Residue(label=None, auth=ResidueAuth(chain='A', number=5, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=14, icode=None, name='C'))
A.G5
A.C14
True Residue(label=None, auth=ResidueAuth(chain='A', number=6, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=13, icode=None, name='C'))
A.G6
A.C13
False Residue(label=None, auth=ResidueAuth(chain='A', number=6, icode=None, name='G')) Residue(label=None, auth=Res

In [6]:
from Bio.PDB import *

residues = {}
# Read the pdb file
parser = PDBParser()
structure = parser.get_structure('pdb', pdb_path)
for residue in structure.get_residues():
    # print(f"{residue.get_segid().strip()}.{residue.get_resname()}{residue.get_id()[1]}")
    # print(next(residue.get_atoms()).get_coord())
    res = f"{residue.get_full_id()[2]}.{residue.get_resname()}{residue.get_id()[1]}"
    
    if res in interest_res:
        print(res)
        residues[res] = residue

A.G3
A.C4
A.G5
A.G6
A.C7
A.C9
A.C10
A.G11
A.C13
A.C14
A.G15
A.C16
A.C26
A.G27
A.G28


/data/miniconda3/envs/difflinker/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 574
  warnings.warn(


In [8]:
def save_as_new_pdb(residues, output_file):
    structure = Structure.Structure('pdb')
    model = Model.Model(0)
    chain = Chain.Chain('A')
    model.add(chain)
    structure.add(model)

    # add the residues to the structure
    for residue in residues:
        chain.add(residue)

    # write the new pdb file
    io = PDBIO()
    io.set_structure(structure)
    io.save(output_file)

for i, bps in enumerate(interest_bps):
    print(bps)
    sel_res = [residues[bps[0]], residues[bps[1]]]
    save_as_new_pdb(sel_res, f"my_out{i}.pdb")

('A.G3', 'A.C16')
('A.C4', 'A.G15')
('A.G5', 'A.C14')
('A.G6', 'A.C13')
('A.C7', 'A.G11')
('A.C9', 'A.G28')
('A.C10', 'A.G27')
('A.G11', 'A.C26')


In [32]:
%autoreload 2
from data_processing import pdb_transform, get_one_hot

In [28]:
transformed = pdb_transform(sel_res, pdb_path)


In [39]:
# save transformed to *.pt file
import torch
name = 'rna_v1_train.pt'
# torch.save([transformed], name)

In [41]:
import torch
pt_data = torch.load(name)
pt_data[0].keys()

dict_keys(['uuid', 'name', 'positions', 'one_hot', 'fragment_mask', 'linker_mask', 'num_atoms', 'anchors', 'charges'])

In [44]:
for a in pt_data:
    print(a['name'])

7WKP_1_B_0
7WKP_1_B_1
7WKP_1_B_2


In [ ]:
atoms = get_atoms(sel_res)
one_hot = get_one_hot(sel_res)
coding = {}
idx_atom = {}
simple_idx_atom = {}
for a, oh in zip(atoms, one_hot):
    print(a, np.argmax(oh))
    coding[a[0]] = np.argmax(oh)
    idx_atom[np.argmax(oh)] = a[0]
    simple_idx_atom[np.argmax(oh)] = a[0]

P 3
C5' 0
O5' 2
C4' 0
O4' 2
C3' 0
O3' 2
C2' 0
O2' 2
C1' 0
N1 1
C2 0
O2 2
N3 1
C4 0
O4 2
C5 0
C6 0
OP1 2
OP2 2
P 3
C5' 0
O5' 2
C4' 0
O4' 2
C3' 0
O3' 2
C2' 0
O2' 2
C1' 0
N1 1
C2 0
O2 2
N3 1
C4 0
O4 2
C5 0
C6 0
OP1 2
OP2 2


In [ ]:
simple_idx_atom

{3: 'P', 0: 'C', 2: 'O', 1: 'N'}

In [ ]:
coding

{'P': 3, 'C': 0, 'O': 2, 'N': 1}